In [1]:
import math
import pandas as pd
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

submission = pd.read_csv('data/sample_submission.csv')

In [2]:
train_data, train_target = train.drop(['index', 'INVC_CONT'],axis=1), train['INVC_CONT']
test_data = test.drop(['index'],axis=1)

In [3]:
data = pd.concat([train_data, test_data])
data

,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,1.129000e+15,5.011000e+15,패션의류,상의
1,1.135000e+15,5.011000e+15,생활/건강,반려동물
2,1.135000e+15,5.011000e+15,패션의류,기타패션의류
3,1.154500e+15,5.011000e+15,식품,농산물
4,1.165000e+15,5.011000e+15,식품,가공식품
...,...,...,...,...
4635,5.013000e+15,4.725000e+15,식품,농산물
4636,5.013000e+15,2.826000e+15,식품,농산물
4637,5.013000e+15,4.311100e+15,식품,농산물
4638,5.013000e+15,4.145000e+15,식품,농산물


In [4]:
data_one = pd.get_dummies(data)

data_one['SEND_CODE'] = data_one['SEND_SPG_INNB'].apply(lambda x: int(str(x)[:4]))
data_one['REC_CODE'] = data_one['REC_SPG_INNB'].apply(lambda x: int(str(x)[:4]))
data_one = data_one.drop(['SEND_SPG_INNB','REC_SPG_INNB'], axis=1)

train_data, test_data = data_one.iloc[:-len(test_data)], data_one.iloc[-len(test_data):]

In [5]:
train_data

,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,...,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산,SEND_CODE,REC_CODE
0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1129,5011
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1135,5011
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1135,5011
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1154,5011
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1165,5011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5011,2635
31996,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5011,1168
31997,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5011,4119
31998,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5011,1132


In [6]:
test_data

,DL_GD_LCLS_NM_디지털/가전,DL_GD_LCLS_NM_생활/건강,DL_GD_LCLS_NM_식품,DL_GD_LCLS_NM_여행/문화,DL_GD_LCLS_NM_패션의류,DL_GD_LCLS_NM_화장품/미용,DL_GD_MCLS_NM_가공식품,DL_GD_MCLS_NM_건강식품,DL_GD_MCLS_NM_건강용품,DL_GD_MCLS_NM_과자,...,DL_GD_MCLS_NM_상의,DL_GD_MCLS_NM_생활용품,DL_GD_MCLS_NM_수산,DL_GD_MCLS_NM_스킨케어,DL_GD_MCLS_NM_음료,DL_GD_MCLS_NM_음반,DL_GD_MCLS_NM_주방용품,DL_GD_MCLS_NM_축산,SEND_CODE,REC_CODE
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,1165
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,1154
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,4139
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,4221
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,2726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,4725
4636,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,2826
4637,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,4311
4638,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5013,4145


In [7]:
# codes = pd.read_csv('data/TC_NU_SPG_1000_METER.csv')
# codes['CODE'] = codes['격자공간고유번호'].apply(lambda x:str(x)[:5])
# codes.head()

In [8]:
# region = pd.read_csv('data/entrc_jeju.txt', sep='|', encoding='euc-kr')
# region_columns = [i for i in range(1,19)]
# drop_columns = [1,3,4,5,7,10,11,12,13]

# region = pd.DataFrame(region.values, columns=region_columns)
# region = region[region[1]==50110]
# region.drop(axis=1, columns=drop_columns, inplace=True)
# region

In [9]:
def rmse(y_true, y_pred):
    return math.sqrt(sum((y_true-y_pred)**2) / len(y_true))

rmse = make_scorer(rmse, greater_is_better=False)

In [10]:
tpot_params = dict({
    'generations':10,
    'population_size':100,
    'scoring': rmse,
    'subsample':1,
    'verbosity':2,
    'random_state':42,
    'n_jobs':-1,
    'use_dask':True,
    'warm_start':True,
    'memory': './tpot_memory/',
    # 'memory':'auto',
    # 'config_dict':'TPOT sparse',
    'periodic_checkpoint_folder':'./tpot_checkpoint/'
})

In [11]:
tpot = TPOTRegressor(**tpot_params)
for i in range(100):
    tpot.fit(train_data, train_target)
    tpot.export(f'tpot_best_pareto/tpot_pipeline_new_{i}.py')
    results = tpot.predict(test_data)
    submission['INVC_CONT'] = results
    submission.to_csv(f'submission/autosk_v{i}.csv', index=False)


Generation 1 - Current best internal CV score: -5.374230447735577

Generation 2 - Current best internal CV score: -5.374230447735577

Generation 3 - Current best internal CV score: -5.374230447735577

Generation 4 - Current best internal CV score: -5.370165987506705

Generation 5 - Current best internal CV score: -5.339445731428144

Generation 6 - Current best internal CV score: -5.339445731428144

Generation 7 - Current best internal CV score: -5.339445731428144

Generation 8 - Current best internal CV score: -5.339445731428144

Generation 9 - Current best internal CV score: -5.339445731428144

Generation 10 - Current best internal CV score: -5.338644290278367

Best pipeline: AdaBoostRegressor(CombineDFs(LinearSVR(MaxAbsScaler(input_matrix), C=1.0, dual=False, epsilon=0.0001, loss=squared_epsilon_insensitive, tol=0.001), input_matrix), learning_rate=0.01, loss=square, n_estimators=100)


c:\Users\Kim\venv\automl\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but MaxAbsScaler was fitted with feature names
  warnings.warn(



Generation 1 - Current best internal CV score: -5.324744664287119

Generation 2 - Current best internal CV score: -5.324744664287119

Generation 3 - Current best internal CV score: -5.324744664287119

Generation 4 - Current best internal CV score: -5.324744664287119

Generation 5 - Current best internal CV score: -5.324744664287119

Generation 6 - Current best internal CV score: -5.324744664287119

Generation 7 - Current best internal CV score: -5.324744664287119

Generation 8 - Current best internal CV score: -5.324744664287119

Generation 9 - Current best internal CV score: -5.324744664287119

Generation 10 - Current best internal CV score: -5.324744664287119

Best pipeline: AdaBoostRegressor(MaxAbsScaler(LassoLarsCV(StandardScaler(input_matrix), normalize=False)), learning_rate=0.01, loss=square, n_estimators=100)

Generation 1 - Current best internal CV score: -5.324744664287119

Generation 2 - Current best internal CV score: -5.324744664287119

Generation 3 - Current best interna

c:\Users\Kim\venv\automl\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(



Generation 1 - Current best internal CV score: -5.309077962709935

Generation 2 - Current best internal CV score: -5.309077962709935

Generation 3 - Current best internal CV score: -5.309077962709935

Generation 4 - Current best internal CV score: -5.309077962709935

Generation 5 - Current best internal CV score: -5.306808763930971

Generation 6 - Current best internal CV score: -5.306808763930971

Generation 7 - Current best internal CV score: -5.306808763930971

Generation 8 - Current best internal CV score: -5.3067429320238775

Generation 9 - Current best internal CV score: -5.3067429320238775

Generation 10 - Current best internal CV score: -5.3067429320238775

Best pipeline: AdaBoostRegressor(StandardScaler(CombineDFs(GradientBoostingRegressor(input_matrix, alpha=0.8, learning_rate=0.001, loss=lad, max_depth=8, max_features=0.7500000000000001, min_samples_leaf=17, min_samples_split=10, n_estimators=100, subsample=0.5), MinMaxScaler(GradientBoostingRegressor(StandardScaler(input_m

c:\Users\Kim\venv\automl\lib\site-packages\sklearn\base.py:445: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(



Generation 1 - Current best internal CV score: -5.305781894841796

Generation 2 - Current best internal CV score: -5.305781894841796

Generation 3 - Current best internal CV score: -5.305781894841796

Generation 4 - Current best internal CV score: -5.305781894841796
Optimization Progress:  46%|████▌     | 504/1100 [2:34:16<4:05:34, 24.72s/pipeline]